In [70]:
from math import exp, sqrt
from statistics import mean
from itertools import cycle


# Assumptions 
# - 2x farm where one asset is stablecoin, e.g.  ETH/USDC
# - there is always borrow availability
# - borrow rate, trading fees, rewards are all constant


class Agent():
    
    def __init__(self, equity, price, borrow_rate, farming_rate):
        print("init")
        
        self.last_price = price
        
        self.borrow_rate = borrow_rate
        self.farming_rate = farming_rate
        
        # Make an intial investment - 50% stablecoin, 50% borrowed shitcoin and receive some amount of LP tokens
        self.equity = equity
        self.assetA_count = equity / price  # shitcoin
        self.assetB_count = equity          # stablecoin
        self.debt_value =  self.assetA_count * price
        self.lp_multiplier = 1
        
        self.upper = price * 1.1
        self.lower = price * 0.9
        
    def apr_to_hour(apr):
        return apr / 365 / 24
    
    def apr_to_day(apr):
        return apr / 365
        
    def rebalance(self):
        print("rebalance")
        
        # update self.debt_value
        
    
    def action(self, new_price):
        print ("\n")
        print ("ACTION...  price is " + str(new_price))
        
        # Calculate pnl
        
        self.price_change_ratio = new_price / self.last_price   # TODO handle div by 0
        self.last_price = new_price
        self.lp_multiplier = sqrt(self.price_change_ratio)
        
        # Taken from https://docs.google.com/spreadsheets/d/1NYVm1aj8cpMOvlsv4Y_JmTrO7X59QnJnH2jDSfZM2zg
        position_value = (2*self.equity*self.lp_multiplier) + 2*mean([self.equity*self.lp_multiplier, self.equity])*(exp(self.farming_rate)-1)
        
        #print ("1: " + str(2*self.equity*self.lp_multiplier))
        #print ("2: " + str(mean([2*self.equity*self.lp_multiplier, 2*self.equity]) * (exp(self.farming_rate)-1)))
        #print ("3: " + str(exp(self.farming_rate)-1))
        
        debt_value = self.assetA_count * new_price * exp(self.borrow_rate)
        equity = position_value - debt_value
        
        print ("price_change_ratio=" + str(self.price_change_ratio))
        print ("lp_multiplier=" + str(self.lp_multiplier))
        print ("position_value=" + str(position_value))
        print ("debt_value=" + str(debt_value))
        print ("equity_value=" + str(equity))
        
        
        
        # Rebalance if necessary
     #   if ((price > self.upper) or (price < self.lower)):
     #       self.rebalance()
     #       self.calculate_position()
            
    
    def calculate_position(self):
        print("calculate_pnl")
        
        # equity
        # rewards

        

# RUN SIMULATION

price = [13.70, 13.70, 14.385, 15.07, 15.76, 11.65, 12.33]

#price = [13.70, 14.39]
#TODO scale rates


licycle = cycle(price)


agent = Agent(
    10000,         # initial equity
    price.pop(0),      # initial token_price 
    0.085,         # borrow rate per time step 
    0.10         # farming rate at 1x leverage per time step
)


for p in price:
    agent.action(p) 

#while p := price.pop():
   #agent.action(p) 
    


# UNUSED
# entry_fee = 0
# exit_fee = 0
# controller_fee = 0.001  
# reinvestent_fee = 0.039 
# tx_fee = 0  # network fee
# performance_fee = 0



#TODO
# repeated iterations of action - calcs are working for 1 step but need to update state - OR only update state on rebalance when you crystalise loss, at other times keep track of timesteps since last rebalance to calculate equity etc.
# put in real price data
# do rebalances
            


init


ACTION...  price is 13.7
price_change_ratio=1.0
lp_multiplier=1.0
position_value=22103.418361512955
debt_value=10887.170666983988
equity_value=11216.247694528967


ACTION...  price is 14.385
price_change_ratio=1.05
lp_multiplier=1.02469507659596
position_value=22623.29193220761
debt_value=11431.529200333187
equity_value=11191.762731874422


ACTION...  price is 15.07
price_change_ratio=1.0476190476190477
lp_multiplier=1.023532631438318
position_value=22598.82047481035
debt_value=11975.887733682388
equity_value=10622.932741127961


ACTION...  price is 15.76
price_change_ratio=1.0457863304578632
lp_multiplier=1.0226369494878733
position_value=22579.964838871136
debt_value=12524.21968698304
equity_value=10055.745151888095


ACTION...  price is 11.65
price_change_ratio=0.7392131979695432
lp_multiplier=0.8597750856878461
position_value=19151.444245516996
debt_value=9258.068486887845
equity_value=9893.37575862915


ACTION...  price is 12.33
price_change_ratio=1.0583690987124463
lp_mult